In [1]:
# detect
# project
# cluster

In [50]:
!nvidia-smi

Tue Mar 11 14:33:38 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:21:00.0  On |                  N/A |
| 28%   36C    P8              21W / 260W |   7587MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
1 + 1

2

In [2]:
from salad.serialize import read

In [5]:
detections = read("/epyc/projects/salad/search/tno_search/DEEP/20190403/A0c/detector_3/snr_5.0/regular/catalog.pkl")

In [6]:
X = detections.X()

In [7]:
# directions
from salad.directions import SearchDirections
import astropy.units as u

dx = 7.98314077571659 * u.arcsec
dt = (X[:, 2].max() - X[:, 2].min())*u.day
ref_time = X[:, 2].min()
directions = SearchDirections([0.1 * u.deg/u.day, 0.5 * u.deg/u.day], [0 * u.deg, 359 * u.deg], dx, dt)

In [43]:
X.shape

(9118, 4)

In [42]:
directions.b.shape

(4848, 2)

In [9]:
# project
import numba
from numba import cuda
import torch
import numpy as np

@numba.njit()
def project_xyz(x: np.float64, y: np.float64, t: np.float64, vx: np.float64, vy: np.float64, reference_time: np.float64):
    x_prime = x - vx * (t - reference_time)
    y_prime = y - vy * (t - reference_time)
    return x_prime, y_prime

@numba.njit()
def digitize_point(x, y, min_x, min_y, dx, dy):
    return int((x - min_x)/dx), int((y - min_y)/dy)

# @numba.njit(parallel=True)
def vote_points_cpu_mask(hough: np.ndarray, X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time, coef, results):
    """
    Given detections vote in the hough space
    """
    n, d = X.shape
    num_dir = directions.shape[0]
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    for j in range(results.shape[0]):
                        mask_dir = results[j, 0]
                        mask_x = results[j, 1]
                        mask_y = results[j, 2]
                        print(mask_dir, mask_x, mask_y)
                        if dir_idx == mask_dir and x_idx == mask_x and y_idx == mask_y:
                            hough[dir_idx, x_idx, y_idx] += coef

@numba.njit(parallel=True)
def vote_points_cpu(hough: np.ndarray, X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time, coef: np.float64=1):
    """
    Given detections vote in the hough space
    """
    n, d = X.shape
    num_dir = directions.shape[0]
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    hough[dir_idx, x_idx, y_idx] += coef

@cuda.jit
def _vote_points_gpu(hough, X, directions, x_min, y_min, dx, dy, reference_time, coef):
    """
    GPU version of vote_points using CUDA parallelism.
    Each thread processes one (dir_idx, point_idx) pair.
    """
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    point_idx = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    num_dir, n = directions.shape[0], X.shape[0]

    if dir_idx >= num_dir or point_idx >= n:
        return  # Out-of-bounds check

    direction = directions[dir_idx]
    p = X[point_idx]
    x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
    x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)

    if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
        cuda.atomic.add(hough, (dir_idx, x_idx, y_idx), coef)  # Atomic update to avoid race conditions
        
@cuda.jit
def _vote_points_gpu_mask(hough, X, directions, x_min, y_min, dx, dy, reference_time, coef, mask_dir, mask_x, mask_y):
    """
    GPU version of vote_points using CUDA parallelism.
    Each thread processes one (dir_idx, point_idx) pair.
    """
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    point_idx = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    num_dir, n = directions.shape[0], X.shape[0]

    if dir_idx >= num_dir or point_idx >= n:
        return  # Out-of-bounds check

    direction = directions[dir_idx]
    p = X[point_idx]
    x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
    x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)

    if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
        if dir_idx == mask_dir and x_idx == mask_x and y_idx == mask_y:
            cuda.atomic.add(hough, (dir_idx, x_idx, y_idx), coef)  # Atomic update to avoid race conditions

def vote_points_gpu(hough: np.ndarray, X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time, coef: np.float64 = 1, mask_dir=-1, mask_x=-1, mask_y=-1):
    """
    Host function to launch the GPU kernel.
    """
    n, d = X.shape
    num_dir = directions.shape[0]

    # Transfer data to GPU
    d_hough = cuda.to_device(hough)
    d_X = cuda.to_device(X)
    d_directions = cuda.to_device(directions)

    # Configure GPU threads and blocks
    threads_per_block = (16, 16)  # Tunable parameters
    blocks_per_grid = ((num_dir + threads_per_block[0] - 1) // threads_per_block[0], 
                       (n + threads_per_block[1] - 1) // threads_per_block[1])

    # Launch the CUDA kernel
    if mask_dir != -1 and mask_x != -1 and mask_y != -1:
        _vote_points_gpu_mask[blocks_per_grid, threads_per_block](
            d_hough, d_X, d_directions, x_min, y_min, dx, dy, reference_time, coef, mask_dir, mask_x, mask_y
        )
    else:
        _vote_points_gpu[blocks_per_grid, threads_per_block](
            d_hough, d_X, d_directions, x_min, y_min, dx, dy, reference_time, coef
        )

    # Copy back the updated Hough space
    return d_hough.copy_to_host()

    
@numba.njit(parallel=True)
def projected_bounds_cpu(X: np.ndarray, directions: np.ndarray, reference_time: np.float64):
    n, d = X.shape
    num_dir = directions.shape[0]
    min_x_arr = np.full(num_dir, np.inf)
    max_x_arr = np.full(num_dir, -np.inf)
    min_y_arr = np.full(num_dir, np.inf)
    max_y_arr = np.full(num_dir, -np.inf)
    
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                min_x_arr[dir_idx] = min(min_x_arr[dir_idx], x_prime)
                max_x_arr[dir_idx] = max(max_x_arr[dir_idx], x_prime)
                min_y_arr[dir_idx] = min(min_y_arr[dir_idx], y_prime)
                max_y_arr[dir_idx] = max(max_y_arr[dir_idx], y_prime)

    min_x = np.min(min_x_arr)
    max_x = np.max(max_x_arr)
    min_y = np.min(min_y_arr)
    max_y = np.max(max_y_arr)
    
    return min_x, max_x, min_y, max_y


@cuda.jit
def _projected_bounds_gpu(X, directions, reference_time, min_x_arr, max_x_arr, min_y_arr, max_y_arr):
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    n, d = X.shape
    num_dir = directions.shape[0]
    
    if dir_idx >= num_dir:
        return

    direction = directions[dir_idx]
    
    # Initialize local min/max
    min_x = np.inf
    max_x = -np.inf
    min_y = np.inf
    max_y = -np.inf

    # Iterate over all points
    for i in range(n):
        p = X[i]
        x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
        min_x = min(min_x, x_prime)
        max_x = max(max_x, x_prime)
        min_y = min(min_y, y_prime)
        max_y = max(max_y, y_prime)

    # Store results in global memory
    min_x_arr[dir_idx] = min_x
    max_x_arr[dir_idx] = max_x
    min_y_arr[dir_idx] = min_y
    max_y_arr[dir_idx] = max_y
    
def projected_bounds_gpu(X: np.ndarray, directions: np.ndarray, reference_time: np.float64):
    n, d = X.shape
    num_dir = directions.shape[0]

    # Allocate device arrays
    d_X = cuda.to_device(X)
    d_directions = cuda.to_device(directions)
    d_min_x_arr = cuda.device_array(num_dir, dtype=np.float64)
    d_max_x_arr = cuda.device_array(num_dir, dtype=np.float64)
    d_min_y_arr = cuda.device_array(num_dir, dtype=np.float64)
    d_max_y_arr = cuda.device_array(num_dir, dtype=np.float64)

    # Define CUDA thread/block config
    threads_per_block = 128
    blocks_per_grid = (num_dir + threads_per_block - 1) // threads_per_block

    # Launch kernel
    _projected_bounds_gpu[blocks_per_grid, threads_per_block](
        d_X, d_directions, reference_time, d_min_x_arr, d_max_x_arr, d_min_y_arr, d_max_y_arr
    )

    # Copy results back to host
    min_x_arr = d_min_x_arr.copy_to_host()
    max_x_arr = d_max_x_arr.copy_to_host()
    min_y_arr = d_min_y_arr.copy_to_host()
    max_y_arr = d_max_y_arr.copy_to_host()

    # Final reduction on CPU
    min_x = np.min(min_x_arr)
    max_x = np.max(max_x_arr)
    min_y = np.min(min_y_arr)
    max_y = np.max(max_y_arr)

    return min_x, max_x, min_y, max_y

@numba.njit(parallel=True)
def hough_argmax_cpu(hough: np.ndarray):
    """
    Finds the location of the maximum value in the Hough space using parallel execution.
    """
    num_dir, x_dim, y_dim = hough.shape
    max_vals = np.full(num_dir, -np.inf)  # Per-direction max values
    max_indices = np.full((num_dir, 3), -1, dtype=np.int32)  # Store (dir_idx, x_idx, y_idx)

    # Parallel over directions
    for dir_idx in numba.prange(num_dir):
        local_max = -np.inf
        local_x_idx = -1
        local_y_idx = -1

        for x_idx in range(x_dim):
            for y_idx in range(y_dim):
                if hough[dir_idx, x_idx, y_idx] > local_max:
                    local_max = hough[dir_idx, x_idx, y_idx]
                    local_x_idx = x_idx
                    local_y_idx = y_idx

        # Store per-thread results
        max_vals[dir_idx] = local_max
        max_indices[dir_idx, 0] = dir_idx
        max_indices[dir_idx, 1] = local_x_idx
        max_indices[dir_idx, 2] = local_y_idx

    # Final reduction: Find the overall max from the per-direction max values
    global_max = -np.inf
    global_max_idx = np.array([-1, -1, -1], dtype=np.int32)

    for i in range(num_dir):
        if max_vals[i] > global_max:
            global_max = max_vals[i]
            global_max_idx[0] = max_indices[i, 0]
            global_max_idx[1] = max_indices[i, 1]
            global_max_idx[2] = max_indices[i, 2]

    return global_max_idx, global_max

@cuda.jit
def _hough_argmax_gpu(hough, max_idx, max_val):
    """
    CUDA kernel to find the max value and its index in the Hough space.
    Uses shared memory and block-wide reduction.
    """
    # Thread IDs
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    x_idx = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    y_idx = cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z

    # Shared memory for per-block reduction
    shared_max_val = cuda.shared.array(shape=(1,), dtype=numba.float64)
    shared_max_idx = cuda.shared.array(shape=(3,), dtype=numba.int32)

    # Initialize shared memory
    if cuda.threadIdx.x == 0 and cuda.threadIdx.y == 0 and cuda.threadIdx.z == 0:
        shared_max_val[0] = -np.inf
        shared_max_idx[0] = -1  # dir_idx
        shared_max_idx[1] = -1  # x_idx
        shared_max_idx[2] = -1  # y_idx
    cuda.syncthreads()

    # Ensure within bounds
    if dir_idx >= hough.shape[0] or x_idx >= hough.shape[1] or y_idx >= hough.shape[2]:
        return

    val = hough[dir_idx, x_idx, y_idx]

    # Block-wide reduction to find max value
    cuda.syncthreads()
    if val > shared_max_val[0]:  
        shared_max_val[0] = val
        shared_max_idx[0] = dir_idx
        shared_max_idx[1] = x_idx
        shared_max_idx[2] = y_idx
    cuda.syncthreads()

    # Only one thread writes to global memory
    if cuda.threadIdx.x == 0 and cuda.threadIdx.y == 0 and cuda.threadIdx.z == 0:
        cuda.atomic.max(max_val, 0, shared_max_val[0])
        if shared_max_val[0] == max_val[0]:  # Ensure atomicity
            max_idx[0] = shared_max_idx[0]
            max_idx[1] = shared_max_idx[1]
            max_idx[2] = shared_max_idx[2]

def hough_argmax_gpu(hough: np.ndarray):
    """
    Wrapper function to execute the CUDA kernel and return the max location.
    """
    d_hough = cuda.to_device(hough)
    d_max_idx = cuda.device_array(3, dtype=np.int32)
    d_max_val = cuda.device_array(1, dtype=np.float64)

    threads_per_block = (8, 8, 8)  # Tunable parameter
    blocks_per_grid = (
        (hough.shape[0] + threads_per_block[0] - 1) // threads_per_block[0],
        (hough.shape[1] + threads_per_block[1] - 1) // threads_per_block[1],
        (hough.shape[2] + threads_per_block[2] - 1) // threads_per_block[2],
    )

    # Launch kernel
    _hough_argmax_gpu[blocks_per_grid, threads_per_block](d_hough, d_max_idx, d_max_val)

    # Copy results back
    max_idx = d_max_idx.copy_to_host()
    max_val = d_max_val.copy_to_host()

    return tuple(max_idx), max_val[0]


In [10]:
%%timeit
min_x, max_x, min_y, max_y = projected_bounds_gpu(X, directions.b, ref_time)

/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 38 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


4.7 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
min_x, max_x, min_y, max_y = projected_bounds_cpu(X, directions.b, ref_time)

95.9 ms ± 66.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
min_x, max_x, min_y, max_y = projected_bounds_gpu(X, directions.b, ref_time)
print(min_x, max_x, min_y, max_y)

216.53093121715725 217.00136276871302 -11.222229046885362 -10.911972249328436


In [13]:
min_x, max_x, min_y, max_y = projected_bounds_cpu(X, directions.b, ref_time)
print(min_x, max_x, min_y, max_y)

216.53093121715725 217.00136276871302 -11.222229046885362 -10.911972249328436


In [14]:
min_x, max_x, min_y, max_y = projected_bounds_gpu(X, directions.b, ref_time)

num_dir = directions.b.shape[0]
_dx = dx.to(u.deg).value
_dy = _dx
num_x = int((max_x - min_x) / _dx  + 1)
num_y = int((max_y - min_y) / _dy  + 1)

hough = np.zeros((num_dir, num_x, num_y), dtype=np.uint32)

In [15]:
# vote_points()
hough.shape

(4848, 213, 140)

In [16]:
%%time
hough2 = vote_points_gpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time)

CPU times: user 207 ms, sys: 75.8 ms, total: 283 ms
Wall time: 373 ms


In [17]:
%%time
vote_points_cpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time)

CPU times: user 751 ms, sys: 40.1 ms, total: 791 ms
Wall time: 789 ms


In [18]:
(hough2 == hough).all()

True

In [19]:
from numba import cuda

@cuda.jit
def _hough_max_gpu(result, hough):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    
    if i >= hough.shape[0]:
        return
    
    y = -1
    z = -1
    v = -np.inf
    for j in range(hough.shape[1]):
        for k in range(hough.shape[2]):
            val = hough[i, j, k]
            if val > v:
                v = val
                y = j
                z = k
    
    result[i][0] = y
    result[i][1] = z
    result[i][2] = v

def hough_max_gpu(hough):
    result = np.zeros((hough.shape[0], 3), dtype=np.int32)
    _hough_max_gpu[512, hough.shape[0] // 512 + 1](result, hough)
    i = result[:, 2].argmax()
    j = result[i, 0]
    k = result[i, 1]
    v = result[i, 2]
    return (i, j, k), v

In [111]:
h = hough.copy()
idx, val = hough_max_gpu(h)
print(idx, val)
h = vote_points_gpu(
    h, X, directions.b, min_x, min_y, _dx, _dy, ref_time, 
    coef=-1, mask_dir=idx[0], mask_x=idx[1], mask_y=idx[2]
)
idx, val = hough_max_gpu(h)
print(idx, val)

/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


(2085, 155, 79) 101
(2580, 130, 64) 99


/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [114]:
# I actually need to loop through the results and mask over all of those values...
def find_clusters_gpu(hough, n=10):
    h = hough.copy()
    results = []
    for i in range(n):
        idx, val = hough_max_gpu(h)
        h = vote_points_gpu(
            h, X, directions.b, min_x, min_y, _dx, _dy, ref_time, 
            coef=-1, mask_dir=idx[0], mask_x=idx[1], mask_y=idx[2]
        )
        results.append((idx, val))
    return results

def search_gpu(X, directions, dx, reference_time, num_results=10):
    n = X.shape[0]
    
    x_min, x_max, y_min, y_max = projected_bounds_gpu(X, directions.b, reference_time)
    
    num_dir = directions.b.shape[0]
    _dx = dx.to(u.deg).value
    _dy = _dx
    num_x = int((x_max - x_min) / _dx  + 1)
    num_y = int((y_max - y_min) / _dy  + 1)

    hough = np.zeros((num_dir, num_x, num_y), dtype=np.uint32)    

    num_dir, num_x, num_y = hough.shape
    
    h = hough.copy()
    
    d_hough = cuda.to_device(hough)
    d_X = cuda.to_device(X)
    d_directions = cuda.to_device(directions.b)
    d_max = cuda.to_device(np.zeros((hough.shape[0], 3), dtype=np.int32))
    d_results = cuda.to_device(np.zeros((num_results, 4), dtype=np.int32))

    def _vote(coef=1, mask_dir=-1, mask_x=-1, mask_y=-1):
        # Configure GPU threads and blocks
        threads_per_block = (16, 16)  # Tunable parameters
        blocks_per_grid = ((num_dir + threads_per_block[0] - 1) // threads_per_block[0], 
                           (n + threads_per_block[1] - 1) // threads_per_block[1])

        # Launch the CUDA kernel
        if mask_dir != -1 and mask_x != -1 and mask_y != -1:
            _vote_points_gpu_mask[blocks_per_grid, threads_per_block](
                d_hough, d_X, d_directions, x_min, y_min, _dx, _dy, reference_time, coef, mask_dir, mask_x, mask_y
            )
        else:
            _vote_points_gpu[blocks_per_grid, threads_per_block](
                d_hough, d_X, d_directions, x_min, y_min, _dx, _dy, reference_time, coef
            )
    
    def _max():
        _hough_max_gpu[256, num_dir // 256 + 1](d_max, d_hough)
        
        
    _vote()
    # results = [] # cuda device array
    for n_i in range(num_results):
        _max()
        i = -1
        v = -np.inf
        for _ in range(len(d_max)):
            if d_max[_, 2] > v:
                v = d_max[_, 2]
                i = _
        j = d_max[i, 0]
        k = d_max[i, 1]
        d_results[n_i, 0] = i
        d_results[n_i, 1] = j
        d_results[n_i, 2] = k
        d_results[n_i, 3] = v
        print("cluster has value", v, "at", (i, j, k))
        _vote(coef=-1, mask_dir=i, mask_x=j, mask_y=k)
    
    return d_results.copy_to_host()


array([[ 2.16620738e+02, -1.10195710e+01,  5.85773073e+04,
         8.45925000e+05],
       [ 2.16620614e+02, -1.10503847e+01,  5.85773073e+04,
         8.45925000e+05],
       [ 2.16624234e+02, -1.10681670e+01,  5.85773073e+04,
         8.45925000e+05],
       ...,
       [ 2.16879090e+02, -1.10967022e+01,  5.85773227e+04,
         8.45934000e+05],
       [ 2.16884674e+02, -1.11136933e+01,  5.85773227e+04,
         8.45934000e+05],
       [ 2.16912418e+02, -1.10304324e+01,  5.85773227e+04,
         8.45934000e+05]])

In [115]:
%%time
search_gpu(X, directions, 7.98314077571659*u.arcsec, ref_time, num_results=20)

cluster has value 101 at (2085, 155, 79)
cluster has value 99 at (2580, 130, 64)
cluster has value 99 at (2710, 171, 50)
cluster has value 99 at (2712, 70, 95)
cluster has value 97 at (2644, 171, 50)
cluster has value 96 at (2778, 146, 68)
cluster has value 94 at (2645, 86, 88)
cluster has value 91 at (2578, 139, 96)
cluster has value 90 at (2456, 151, 60)
cluster has value 90 at (2579, 139, 96)
cluster has value 90 at (2847, 146, 68)
cluster has value 84 at (2642, 136, 67)
cluster has value 84 at (2646, 70, 96)
cluster has value 83 at (2577, 136, 67)
cluster has value 83 at (2644, 87, 88)
cluster has value 82 at (2643, 139, 96)
cluster has value 82 at (2644, 139, 96)
cluster has value 81 at (2646, 86, 88)
cluster has value 80 at (2641, 136, 67)
cluster has value 79 at (2576, 136, 67)
CPU times: user 4.22 s, sys: 743 ms, total: 4.96 s
Wall time: 4.96 s


array([[2085,  155,   79,  101],
       [2580,  130,   64,   99],
       [2710,  171,   50,   99],
       [2712,   70,   95,   99],
       [2644,  171,   50,   97],
       [2778,  146,   68,   96],
       [2645,   86,   88,   94],
       [2578,  139,   96,   91],
       [2456,  151,   60,   90],
       [2579,  139,   96,   90],
       [2847,  146,   68,   90],
       [2642,  136,   67,   84],
       [2646,   70,   96,   84],
       [2577,  136,   67,   83],
       [2644,   87,   88,   83],
       [2643,  139,   96,   82],
       [2644,  139,   96,   82],
       [2646,   86,   88,   81],
       [2641,  136,   67,   80],
       [2576,  136,   67,   79]], dtype=int32)

In [107]:
import numpy as np
import numba

def hough_max_cpu(hough):
    idx = np.unravel_index(hough.argmax(), hough.shape)
    return idx, hough[idx]

@numba.njit(parallel=True)
def make_bins_cpu(X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time):
    n, d = X.shape
    num_dir = directions.shape[0]
    bins = np.full((num_dir, n, 2), -1)
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)
                bins[dir_idx, i, 0] = x_idx
                bins[dir_idx, i, 1] = y_idx
    return bins

@numba.njit(parallel=True)
def vote_points_cpu(hough: np.ndarray, X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time, coef):
    n, d = X.shape
    num_dir = directions.shape[0]
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    hough[dir_idx, x_idx, y_idx] += coef

@numba.njit(parallel=True)
def vote_bins_cpu(hough: np.ndarray, bins: np.ndarray, directions: np.ndarray, coef):
    _, n, d = bins.shape
    num_dir = directions.shape[0]
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                x_idx = bins[dir_idx, i, 0]
                y_idx = bins[dir_idx, i, 1]
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    hough[dir_idx, x_idx, y_idx] += coef
    
@numba.njit(parallel=True)
def find_voters_bins_cpu(hough: np.ndarray, bins: np.ndarray, directions: np.ndarray, mask_dir, mask_x, mask_y):
    _, n, d = bins.shape
    num_dir = directions.shape[0]
    
    mask = np.full((num_dir, n), False)
    _mask = np.full(n, False)
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                x_idx = bins[dir_idx, i, 0]
                y_idx = bins[dir_idx, i, 1]                
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    if dir_idx == mask_dir and x_idx == mask_x and y_idx == mask_y:
                        mask[dir_idx, i] = True
    for i in range(n):
        for dir_idx in range(num_dir):
            _mask[i] |= mask[dir_idx, i]
            
    return _mask

@numba.njit(parallel=True)
def find_voters_points_cpu(hough: np.ndarray, X: np.ndarray, directions: np.ndarray, x_min, y_min, dx, dy, reference_time, mask_dir, mask_x, mask_y):
    n, d = X.shape
    num_dir = directions.shape[0]
    
    mask = np.full((num_dir, n), False)
    _mask = np.full(n, False)
    for dir_idx in numba.prange(num_dir): # for each direction
        if dir_idx < num_dir:
            direction = directions[dir_idx]
            for i in range(n): # for each point
                p = X[i]
                x_prime, y_prime = project_xyz(p[0], p[1], p[2], direction[0], direction[1], reference_time)
                x_idx, y_idx = digitize_point(x_prime, y_prime, x_min, y_min, dx, dy)
                if 0 <= x_idx < hough.shape[1] and 0 <= y_idx < hough.shape[2]:
                    if dir_idx == mask_dir and x_idx == mask_x and y_idx == mask_y:
                        mask[dir_idx, i] = True
    for i in range(n):
        for dir_idx in range(num_dir):
            _mask[i] |= mask[dir_idx, i]
            
    return _mask

def find_clusters_points_cpu(X, hough, n=10):
    results = np.full((n, 4), -1)
    include = np.full(X.shape[0], True)
    for i in range(n):
        idx, val = hough_max_cpu(hough)
        print("cluster has value", val, "at", idx)
        voters = find_voters_points_cpu(
            hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time, *idx
        )
        vote_points_cpu(
            hough, X[include & voters], directions.b, min_x, min_y, _dx, _dy, ref_time, -1
        )
        include &= ~voters # exclude voters
        print(include.sum(), "/", X.shape[0], "points remain")
        results[i, 0] = idx[0]
        results[i, 1] = idx[1]
        results[i, 2] = idx[2]
        results[i, 3] = val
        
    return results

def find_clusters_bins_cpu(bins, hough, n=10):
    results = np.full((n, 4), -1)
    include = np.full(bins.shape[1], True)
    for i in range(n):
        idx, val = hough_max_cpu(hough)
        print("cluster has value", val, "at", idx)
        voters = find_voters_bins_cpu(
            hough, bins, directions.b, *idx
        )
        vote_bins_cpu(
            hough, bins[:, include & voters], directions.b, -1
        )
        include &= ~voters # exclude voters
        print(include.sum(), "/", X.shape[0], "points remain")
        results[i, 0] = idx[0]
        results[i, 1] = idx[1]
        results[i, 2] = idx[2]
        results[i, 3] = val
        
    return results

def search_cpu(X, directions, dx, reference_time, num_results=10, precompute=False):
    n = X.shape[0]
    
    x_min, x_max, y_min, y_max = projected_bounds_cpu(X, directions.b, reference_time)
    
    num_dir = directions.b.shape[0]
    _dx = dx.to(u.deg).value
    _dy = _dx
    num_x = int((x_max - x_min) / _dx  + 1)
    num_y = int((y_max - y_min) / _dy  + 1)

    hough = np.zeros((num_dir, num_x, num_y), dtype=np.uint32)    

    num_dir, num_x, num_y = hough.shape
    
    if precompute:
        bins = make_bins_cpu(X, directions.b, min_x, min_y, _dx, _dy, reference_time)
        print(bins.shape)
        vote_bins_cpu(hough, bins, directions.b, 1)
        results = find_clusters_bins_cpu(bins, hough, n=num_results)
    else:
        vote_points_cpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time, 1)
        results = find_clusters_points_cpu(X, hough, n=num_results)
    
    return results
        
# min_x, max_x, min_y, max_y = projected_bounds_cpu(X, directions.b, ref_time)

# num_dir = directions.b.shape[0]
# _dx = dx.to(u.deg).value
# _dy = _dx
# num_x = int((max_x - min_x) / _dx  + 1)
# num_y = int((max_y - min_y) / _dy  + 1)

# hough = np.zeros((num_dir, num_x, num_y), dtype=np.int32)

# vote_points_cpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time, 1)
# h = hough.copy()
# results = find_clusters_cpu(h, n=20)

In [108]:
%%time
search_cpu(X, directions, dx, ref_time, num_results=10, precompute=False)

cluster has value 101 at (2085, 155, 79)
9017 / 9118 points remain
cluster has value 99 at (2580, 130, 64)
8918 / 9118 points remain
cluster has value 99 at (2710, 171, 50)
8819 / 9118 points remain
cluster has value 99 at (2712, 70, 95)
8720 / 9118 points remain
cluster has value 96 at (2778, 146, 68)
8624 / 9118 points remain
cluster has value 94 at (2645, 86, 88)
8530 / 9118 points remain
cluster has value 91 at (2578, 139, 96)
8439 / 9118 points remain
cluster has value 90 at (2456, 151, 60)
8349 / 9118 points remain
cluster has value 84 at (2642, 136, 67)
8265 / 9118 points remain
cluster has value 75 at (2779, 111, 84)
8190 / 9118 points remain
CPU times: user 3.57 s, sys: 65.4 ms, total: 3.64 s
Wall time: 4.19 s


array([[2085,  155,   79,  101],
       [2580,  130,   64,   99],
       [2710,  171,   50,   99],
       [2712,   70,   95,   99],
       [2778,  146,   68,   96],
       [2645,   86,   88,   94],
       [2578,  139,   96,   91],
       [2456,  151,   60,   90],
       [2642,  136,   67,   84],
       [2779,  111,   84,   75]])

In [109]:
%%time
search_cpu(X, directions, dx, ref_time, num_results=10, precompute=True)

(4848, 9118, 2)
cluster has value 101 at (2085, 155, 79)
9017 / 9118 points remain
cluster has value 99 at (2580, 130, 64)
8918 / 9118 points remain
cluster has value 99 at (2710, 171, 50)
8819 / 9118 points remain
cluster has value 99 at (2712, 70, 95)
8720 / 9118 points remain
cluster has value 96 at (2778, 146, 68)
8624 / 9118 points remain
cluster has value 94 at (2645, 86, 88)
8530 / 9118 points remain
cluster has value 91 at (2578, 139, 96)
8439 / 9118 points remain
cluster has value 90 at (2456, 151, 60)
8349 / 9118 points remain
cluster has value 84 at (2642, 136, 67)
8265 / 9118 points remain
cluster has value 75 at (2779, 111, 84)
8190 / 9118 points remain
CPU times: user 3.35 s, sys: 115 ms, total: 3.47 s
Wall time: 3.46 s


array([[2085,  155,   79,  101],
       [2580,  130,   64,   99],
       [2710,  171,   50,   99],
       [2712,   70,   95,   99],
       [2778,  146,   68,   96],
       [2645,   86,   88,   94],
       [2578,  139,   96,   91],
       [2456,  151,   60,   90],
       [2642,  136,   67,   84],
       [2779,  111,   84,   75]])

In [ ]:
# INFO:salad.hough:next cluster has value 101 at (2085, 155, 79)
# INFO:salad.hough:9017 points remain
# INFO:salad.hough:next cluster has value 99 at (2580, 130, 64)
# INFO:salad.hough:8918 points remain
# INFO:salad.hough:next cluster has value 99 at (2710, 171, 50)
# INFO:salad.hough:8819 points remain
# INFO:salad.hough:next cluster has value 99 at (2712, 70, 95)
# INFO:salad.hough:8720 points remain
# INFO:salad.hough:next cluster has value 96 at (2778, 146, 68)
# INFO:salad.hough:8624 points remain
# INFO:salad.hough:next cluster has value 94 at (2645, 86, 88)
# INFO:salad.hough:8530 points remain
# INFO:salad.hough:next cluster has value 91 at (2578, 139, 96)
# INFO:salad.hough:8439 points remain
# INFO:salad.hough:next cluster has value 90 at (2456, 151, 60)
# INFO:salad.hough:8349 points remain
# INFO:salad.hough:next cluster has value 84 at (2642, 136, 67)
# INFO:salad.hough:8265 points remain
# INFO:salad.hough:next cluster has value 75 at (2779, 111, 84)
# INFO:salad.hough:8190 points remain
# INFO:salad.hough:next cluster has value 61 at (3672, 71, 30)
# INFO:salad.hough:8129 points remain
# INFO:salad.hough:next cluster has value 44 at (2710, 82, 35)
# INFO:salad.hough:8085 points remain

In [ ]:
array([[1330,  124,   63,  101],
       [1667,  137,   40,   99],
       [1722,   56,   76,   99],
       [1668,  104,   51,   98],
       [1670,  103,   51,   97],
       [1774,  117,   54,   96],
       [1720,  137,   40,   95],
       [1722,   69,   70,   95],
       [1570,  120,   48,   92],
       [1616,  111,   77,   91]])

In [25]:
%%time
min_x, max_x, min_y, max_y = projected_bounds_gpu(X, directions.b, ref_time)

num_dir = directions.b.shape[0]
_dx = dx.to(u.deg).value
_dy = _dx
num_x = int((max_x - min_x) / _dx  + 1)
num_y = int((max_y - min_y) / _dy  + 1)

hough = np.zeros((num_dir, num_x, num_y), dtype=np.uint32)

hough = vote_points_gpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time)
find_clusters_gpu(hough, n=10)

/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


CPU times: user 2.13 s, sys: 747 ms, total: 2.88 s
Wall time: 2.88 s


[((2085, 155, 79), 101),
 ((2580, 130, 64), 99),
 ((2710, 171, 50), 99),
 ((2712, 70, 95), 99),
 ((2644, 171, 50), 97),
 ((2778, 146, 68), 96),
 ((2645, 86, 88), 94),
 ((2578, 139, 96), 91),
 ((2456, 151, 60), 90),
 ((2579, 139, 96), 90)]

In [26]:
%%time
min_x, max_x, min_y, max_y = projected_bounds_gpu(X, directions.b, ref_time)

num_dir = directions.b.shape[0]
_dx = dx.to(u.deg).value
_dy = _dx
num_x = int((max_x - min_x) / _dx  + 1)
num_y = int((max_y - min_y) / _dy  + 1)

hough = np.zeros((num_dir, num_x, num_y), dtype=np.uint32)
hough = vote_points_gpu(hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time)
hough_max_gpu(hough)

CPU times: user 226 ms, sys: 74.9 ms, total: 301 ms
Wall time: 299 ms


((2085, 155, 79), 101)

In [92]:
%%time
hough_max_cpu(hough)

CPU times: user 16.4 ms, sys: 886 µs, total: 17.3 ms
Wall time: 15.9 ms


((1330, 124, 63), 101)

In [95]:
hough2 = vote_points_gpu(
    hough, X, directions.b, min_x, min_y, _dx, _dy, ref_time,
    coef=-1,
    mask_dir=1330, mask_x=124, mask_y=63
)

In [101]:
hough2.max()

0

In [54]:
# vote with mask...

In [26]:
np.multiply.reduce(hough.shape) // 256 + 1

229076

In [27]:
hough.shape

(3080, 170, 112)

True

In [23]:
%%time
np.unravel_index(hough.argmax(), hough.shape)

CPU times: user 19.4 ms, sys: 21 µs, total: 19.4 ms
Wall time: 17.6 ms


(1330, 124, 63)

In [28]:
%%time
hough_argmax_gpu(hough)

CPU times: user 21.6 ms, sys: 7.95 ms, total: 29.5 ms
Wall time: 28.5 ms


((1664, 108, 53), 100.0)

In [25]:
%%time
hough_argmax_cpu(hough)

CPU times: user 80.6 ms, sys: 906 µs, total: 81.5 ms
Wall time: 79.9 ms


(array([1330,  124,   63], dtype=int32), 202.0)

In [47]:
import numba
from numba import cuda
import numpy as np

@cuda.jit
def find_local_hough_max(hough, local_max_vals, local_max_idxs):
    """
    Step 1: Each CUDA block finds its local maximum value and index.
    """
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    x_idx = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    y_idx = cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z

    # Ensure thread is in valid bounds
    if dir_idx < hough.shape[0] and x_idx < hough.shape[1] and y_idx < hough.shape[2]:
        val = hough[dir_idx, x_idx, y_idx]
        block_id = cuda.blockIdx.x + cuda.blockIdx.y * cuda.gridDim.x + cuda.blockIdx.z * cuda.gridDim.x * cuda.gridDim.y

        # Use atomic operations to find block-wise maximum
        cuda.atomic.max(local_max_vals, block_id, val)

        if local_max_vals[block_id] == val:  # Ensure correct index
            local_max_idxs[block_id, 0] = dir_idx
            local_max_idxs[block_id, 1] = x_idx
            local_max_idxs[block_id, 2] = y_idx

@cuda.jit
def find_global_hough_max(local_max_vals, local_max_idxs, max_val, max_idx):
    """
    Step 2: Find the global maximum value and index.
    """
    thread_id = cuda.threadIdx.x
    num_blocks = local_max_vals.shape[0]

    # Shared memory for block-wide reduction
    shared_max_val = cuda.shared.array(1, numba.float64)
    shared_max_idx = cuda.shared.array(3, numba.int32)

    # Initialize shared memory
    if thread_id == 0:
        shared_max_val[0] = -np.inf
        shared_max_idx[0] = -1
        shared_max_idx[1] = -1
        shared_max_idx[2] = -1
    cuda.syncthreads()

    # Load thread data
    if thread_id < num_blocks:
        val = local_max_vals[thread_id]
        if val > shared_max_val[0]:
            shared_max_val[0] = val
            shared_max_idx[0] = local_max_idxs[thread_id, 0]
            shared_max_idx[1] = local_max_idxs[thread_id, 1]
            shared_max_idx[2] = local_max_idxs[thread_id, 2]
    cuda.syncthreads()

    # Final max update by thread 0
    if thread_id == 0:
        max_val[0] = shared_max_val[0]
        max_idx[0] = shared_max_idx[0]
        max_idx[1] = shared_max_idx[1]
        max_idx[2] = shared_max_idx[2]

def find_hough_max(hough: np.ndarray):
    """
    Wrapper function to execute the two-pass reduction on the GPU.
    """
    d_hough = cuda.to_device(hough)

    # Determine grid and block sizes
    threads_per_block = (8, 8, 8)
    blocks_per_grid = (
        (hough.shape[0] + threads_per_block[0] - 1) // threads_per_block[0],
        (hough.shape[1] + threads_per_block[1] - 1) // threads_per_block[1],
        (hough.shape[2] + threads_per_block[2] - 1) // threads_per_block[2],
    )
    num_blocks = blocks_per_grid[0] * blocks_per_grid[1] * blocks_per_grid[2]
    print(num_blocks, blocks_per_grid)

    # Allocate device memory for local maxima
    d_local_max_vals = cuda.device_array(num_blocks, dtype=np.float64)
    d_local_max_idxs = cuda.device_array((num_blocks, 3), dtype=np.int32)

    # Step 1: Find per-block maxima
    find_local_hough_max[blocks_per_grid, threads_per_block](d_hough, d_local_max_vals, d_local_max_idxs)

    # Allocate device memory for final reduction
    d_max_val = cuda.device_array(1, dtype=np.float64)
    d_max_idx = cuda.device_array(3, dtype=np.int32)

    # Initialize max_val to -inf before reduction
    d_max_val[:] = -np.inf

    # Step 2: Find the global maximum (use enough threads)
    threads_final_reduction = min(1024, num_blocks)
    find_global_hough_max[1, threads_final_reduction](d_local_max_vals, d_local_max_idxs, d_max_val, d_max_idx)

    # Copy results back
    max_idx = d_max_idx.copy_to_host()
    max_val = d_max_val.copy_to_host()

    return tuple(max_idx), max_val[0]

find_hough_max(hough)

118580 (385, 22, 14)


/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


((0, 0, 0), 1.867356296054e-312)

In [62]:
import numba
from numba import cuda
import numpy as np

@cuda.jit
def find_local_hough_max(hough, local_max_vals, local_max_idxs):
    """
    Step 1: Each CUDA block finds its local maximum value and index.
    """
    dir_idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    x_idx = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    y_idx = cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z

    # Ensure thread is in valid bounds
    if dir_idx >= hough.shape[0] or x_idx >= hough.shape[1] or y_idx >= hough.shape[2]:
        return  

    val = hough[dir_idx, x_idx, y_idx]
    block_id = cuda.blockIdx.x + cuda.blockIdx.y * cuda.gridDim.x + cuda.blockIdx.z * cuda.gridDim.x * cuda.gridDim.y

    # Use atomic operations to find block-wise maximum
    cuda.atomic.max(local_max_vals, block_id, val)

    if local_max_vals[block_id] == val:  # Ensure correct index
        local_max_idxs[block_id, 0] = dir_idx
        local_max_idxs[block_id, 1] = x_idx
        local_max_idxs[block_id, 2] = y_idx

@cuda.jit
def reduce_max(local_max_vals, local_max_idxs, reduced_max_vals, reduced_max_idxs):
    """
    Multi-pass reduction: Reduce max values and indices.
    """
    thread_id = cuda.grid(1)
    num_threads = cuda.gridsize(1)

    # Grid-stride loop to handle arbitrary sizes
    max_val = -np.inf
    max_idx = (-1, -1, -1)

    for i in range(thread_id, local_max_vals.shape[0], num_threads):
        if local_max_vals[i] > max_val:
            max_val = local_max_vals[i]
            max_idx = (local_max_idxs[i, 0], local_max_idxs[i, 1], local_max_idxs[i, 2])

    # Store results
    if thread_id < reduced_max_vals.shape[0]:
        reduced_max_vals[thread_id] = max_val
        reduced_max_idxs[thread_id, 0] = max_idx[0]
        reduced_max_idxs[thread_id, 1] = max_idx[1]
        reduced_max_idxs[thread_id, 2] = max_idx[2]

def find_hough_max(hough: np.ndarray):
    """
    Wrapper function for multi-pass reduction.
    """
    d_hough = cuda.to_device(hough)

    # Determine grid and block sizes
    threads_per_block = (8, 8, 8)
    blocks_per_grid = (
        (hough.shape[0] + threads_per_block[0] - 1) // threads_per_block[0],
        (hough.shape[1] + threads_per_block[1] - 1) // threads_per_block[1],
        (hough.shape[2] + threads_per_block[2] - 1) // threads_per_block[2],
    )
    num_blocks = blocks_per_grid[0] * blocks_per_grid[1] * blocks_per_grid[2]
    
    # Allocate device memory for local maxima
    d_local_max_vals = cuda.device_array(num_blocks, dtype=np.int32)
    d_local_max_idxs = cuda.device_array((num_blocks, 3), dtype=np.int32)

    # Step 1: Find per-block maxima
    find_local_hough_max[blocks_per_grid, threads_per_block](d_hough, d_local_max_vals, d_local_max_idxs)

    # Multi-pass reduction
    while num_blocks > 1:
        num_threads = min(1024, num_blocks)
        num_blocks = (num_blocks + num_threads - 1) // num_threads  # Reduce further
        
        d_reduced_max_vals = cuda.device_array(num_blocks, dtype=np.int32)
        d_reduced_max_idxs = cuda.device_array((num_blocks, 3), dtype=np.int32)

        reduce_max[num_blocks, num_threads](d_local_max_vals, d_local_max_idxs, d_reduced_max_vals, d_reduced_max_idxs)

        # Swap buffers for next reduction step
        d_local_max_vals = d_reduced_max_vals
        d_local_max_idxs = d_reduced_max_idxs

    # Copy results back
    max_idx = d_local_max_idxs.copy_to_host()[0]
    max_val = d_local_max_vals.copy_to_host()[0]

    return tuple(max_idx), max_val

arr = (100 * np.random.randn(1000).reshape(10, 10, 10)).astype(int)

find_hough_max(arr)

/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


((7, 3, 7), 294)

In [38]:
import numpy as np
from numba import cuda

@cuda.jit
def block_max_3d(arr, block_max_vals, block_max_idxs):
    """
    Kernel to compute the maximum value and its index within each block.
    """
    tx, ty, tz = cuda.threadIdx.x, cuda.threadIdx.y, cuda.threadIdx.z
    bx, by, bz = cuda.blockIdx.x, cuda.blockIdx.y, cuda.blockIdx.z
    bdx, bdy, bdz = cuda.blockDim.x, cuda.blockDim.y, cuda.blockDim.z
    
    # Compute global index
    x, y, z = bx * bdx + tx, by * bdy + ty, bz * bdz + tz
    
    tid = tx + ty * bdx + tz * (bdx * bdy)
    
    # Dynamically allocate shared memory
    shared_max_val = cuda.shared.array(shape=(256,), dtype=numba.float32)
    shared_max_idx = cuda.shared.array(shape=(256, 3), dtype=numba.int32)
    
    if tid < 256:
        shared_max_val[tid] = -np.inf
        shared_max_idx[tid, 0] = -1
        shared_max_idx[tid, 1] = -1
        shared_max_idx[tid, 2] = -1
    
    cuda.syncthreads()
    
    # Load data and perform local reduction
    if x < arr.shape[0] and y < arr.shape[1] and z < arr.shape[2]:
        val = arr[x, y, z]
        shared_max_val[tid] = val
        shared_max_idx[tid, 0] = x
        shared_max_idx[tid, 1] = y
        shared_max_idx[tid, 2] = z
    
    cuda.syncthreads()
    
    # Perform block-level reduction
    stride = 128
    while stride > 0:
        if tid < stride and tid + stride < 256:
            if shared_max_val[tid + stride] > shared_max_val[tid]:
                shared_max_val[tid] = shared_max_val[tid + stride]
                shared_max_idx[tid, 0] = shared_max_idx[tid + stride, 0]
                shared_max_idx[tid, 1] = shared_max_idx[tid + stride, 1]
                shared_max_idx[tid, 2] = shared_max_idx[tid + stride, 2]
        cuda.syncthreads()
        stride //= 2
    
    # Store block maximums to global memory
    if tid == 0:
        block_idx = bx + by * cuda.gridDim.x + bz * cuda.gridDim.x * cuda.gridDim.y
        block_max_vals[block_idx] = shared_max_val[0]
        block_max_idxs[block_idx, 0] = shared_max_idx[0, 0]
        block_max_idxs[block_idx, 1] = shared_max_idx[0, 1]
        block_max_idxs[block_idx, 2] = shared_max_idx[0, 2]

@cuda.jit
def final_max_reduce(block_max_vals, block_max_idxs, max_val, max_idx):
    """
    Kernel to find the global maximum from the block-level maximums.
    """
    tid = cuda.threadIdx.x
    
    # Dynamically allocate shared memory
    shared_max_val = cuda.shared.array(shape=(256,), dtype=numba.float32)
    shared_max_idx = cuda.shared.array(shape=(256, 3), dtype=numba.int32)
    
    if tid < block_max_vals.shape[0]:
        shared_max_val[tid] = block_max_vals[tid]
        shared_max_idx[tid, 0] = block_max_idxs[tid, 0]
        shared_max_idx[tid, 1] = block_max_idxs[tid, 1]
        shared_max_idx[tid, 2] = block_max_idxs[tid, 2]
    else:
        shared_max_val[tid] = -np.inf
    
    cuda.syncthreads()
    
    # Perform reduction within a single block
    stride = block_max_vals.shape[0] // 2
    while stride > 0:
        if tid < stride:
            if shared_max_val[tid + stride] > shared_max_val[tid]:
                shared_max_val[tid] = shared_max_val[tid + stride]
                shared_max_idx[tid, 0] = shared_max_idx[tid + stride, 0]
                shared_max_idx[tid, 1] = shared_max_idx[tid + stride, 1]
                shared_max_idx[tid, 2] = shared_max_idx[tid + stride, 2]
        cuda.syncthreads()
        stride //= 2
    
    # Store result to global memory
    if tid == 0:
        max_val[0] = shared_max_val[0]
        max_idx[0], max_idx[1], max_idx[2] = shared_max_idx[0, 0], shared_max_idx[0, 1], shared_max_idx[0, 2]

# Example Usage
shape = (256, 256, 256)  # Larger array size for testing
arr = np.random.rand(*shape).astype(np.float32)
d_arr = cuda.to_device(arr)

threads_per_block = (8, 8, 8)
blocks_per_grid = tuple((s + t - 1) // t for s, t in zip(shape, threads_per_block))
num_blocks = blocks_per_grid[0] * blocks_per_grid[1] * blocks_per_grid[2]

d_block_max_vals = cuda.device_array(num_blocks, dtype=np.float32)
d_block_max_idxs = cuda.device_array((num_blocks, 3), dtype=np.int32)
d_max_val = cuda.device_array(1, dtype=np.float32)
d_max_idx = cuda.device_array(3, dtype=np.int32)

# First pass: Block-level maximums
block_max_3d[blocks_per_grid, threads_per_block](d_arr, d_block_max_vals, d_block_max_idxs)

# Second pass: Global maximum reduction
final_max_reduce[1, min(256, num_blocks)](d_block_max_vals, d_block_max_idxs, d_max_val, d_max_idx)

# Synchronize and check for errors
cuda.synchronize()
print(cuda.get_last_error())

# Copy results to host
max_val = d_max_val.copy_to_host()
max_idx = d_max_idx.copy_to_host()

print("Max Value:", max_val[0], "==", arr.max())
print("Max Index:", tuple(max_idx), "==", np.unravel_index(arr.argmax(), arr.shape))


ERROR:numba.cuda.cudadrv.driver:Call to cuMemAlloc results in UNKNOWN_CUDA_ERROR


CudaAPIError: [700] Call to cuMemAlloc results in UNKNOWN_CUDA_ERROR

In [39]:
from numba import cuda

free_mem, total_mem = cuda.current_context().get_memory_info()
print(f"Free Memory: {free_mem / 1e6} MB, Total Memory: {total_mem / 1e6} MB")


ERROR:numba.cuda.cudadrv.driver:Call to cuMemGetInfo results in UNKNOWN_CUDA_ERROR


CudaAPIError: [700] Call to cuMemGetInfo results in UNKNOWN_CUDA_ERROR

In [72]:
!nvidia-smi

Fri Mar  7 00:26:47 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off | 00000000:21:00.0  On |                  N/A |
| 39%   51C    P2              64W / 260W |    669MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [52]:
hough.max()

202

In [211]:
import numpy as np
from numba import cuda

@cuda.jit
def max_3d_array(arr, max_val, max_idx):
    tx, ty, tz = cuda.grid(3)
    
    # Initialize shared memory for local maximum values and indices
    shared_max_val = cuda.shared.array(shape=(1,), dtype=numba.int32)
    shared_max_idx = cuda.shared.array(shape=(3,), dtype=numba.int32)
    
    if tx == 0 and ty == 0 and tz == 0:
        shared_max_val[0] = -np.inf
        shared_max_idx[0] = -1
        shared_max_idx[1] = -1
        shared_max_idx[2] = -1
    
    cuda.syncthreads()
    
    if tx < arr.shape[0] and ty < arr.shape[1] and tz < arr.shape[2]:
        val = arr[tx, ty, tz]
        
        # Use atomic operation for thread safety
        prev = cuda.atomic.max(shared_max_val, 0, val)
        if val > prev:
            shared_max_val = val
            shared_max_idx[0] = tx
            shared_max_idx[1] = ty
            shared_max_idx[2] = tz
    
    cuda.syncthreads()
    
    # Store result to global memory by a single thread
    if tx == 0 and ty == 0 and tz == 0:
        max_val[0] = shared_max_val[0]
        max_idx[0] = shared_max_idx[0]
        max_idx[1] = shared_max_idx[1]
        max_idx[2] = shared_max_idx[2]


# Example Usage
# shape = hough.shape
# shape = (120, 120, 120)
# arr = np.random.rand(*shape).astype(hough.dtype)

shape = hough.shape
arr = hough
d_max_val = cuda.device_array(1, dtype=np.int32)
d_max_idx = cuda.device_array(3, dtype=np.int32)
d_arr = cuda.to_device(arr)

threads_per_block = (8, 8, 8)
blocks_per_grid = tuple((s + t - 1) // t for s, t in zip(shape, threads_per_block))
print(threads_per_block, blocks_per_grid)
print(arr.shape)

max_3d_array[blocks_per_grid, threads_per_block](d_arr, d_max_val, d_max_idx)

max_val = d_max_val.copy_to_host()
max_idx = d_max_idx.copy_to_host()
print("Max Value:", max_val[0])
print("Max Index:", tuple(max_idx))

(8, 8, 8) (385, 22, 14)
(3080, 170, 112)
Max Value: 0
Max Index: (-1, -1, -1)


In [ ]:
@cuda.jit
def cmax(result, array):
    

In [39]:
@cuda.jit
def max_example_3d(result, values):
    """
    Find the maximum value in values and store in result[0].
    Both result and values are 3d arrays.
    """
    i, j, k = cuda.grid(3)
    # Atomically store to result[0,1,2] from values[i, j, k]
    cuda.atomic.max(result, (0, 1, 2), values[i, j, k])

shape = hough.shape
arr = hough
# arr = np.random.rand(np.multiply.reduce(shape)).reshape(shape)
result = np.zeros((3, 3, 3), dtype=arr.dtype)
max_example_3d[128, 16](result, arr)
print(result[0, 1, 2], "==", np.max(arr))
assert(result[0, 1, 2] == np.max(arr))

0 == 202


/nvme/users/stevengs/opt_lsst/conda/envs/lsst-scipipe-8.0.0/lib/python3.11/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


AssertionError: 

In [33]:
arr.shape

(3080, 170, 112)

In [206]:
idx = np.unravel_index(arr.argmax(), arr.shape)
val = arr[idx]
print("Max Value:", val)
print("Max Index:", idx)

Max Value: 707
Max Index: (1330, 124, 63)


In [142]:
arr.max()

1.0